## Name: ??
### Date: 11/02/2025
### Status: Ongoing
### Idea: 
The idea is to guide an embedder based on top-k of same and opossing class with a contrastive loss.

Not sure how this is different than adversarial sampling contrastive loss.

Maybe we can think of it as NX-Ent [check here p.4 bottom left](https://arxiv.org/pdf/2109.00217) but each training batch is essentially a subgraph of the whole dataset, where for each sample (most of) the top-k positive/negatives are found in the batch, so the positive/negative pairs created are not at random, but on its most similar of the same/opposing class.

### Results:
Seems working. I may be hacked though.

Details:
- Interestingly enough some cases where we just use the k-nn on top is better than the classification head.
- Also, for the positive pairs it also works (sometimes better) when working on the least similar data.

In [ ]:
import pandas as pd
import cached_path
from pmlb import fetch_data
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import StratifiedKFold, cross_val_predict
import time
from sklearn.metrics import classification_report, accuracy_score, precision_recall_fscore_support


path_to_data_summary = "https://raw.githubusercontent.com/EpistasisLab/pmlb/master/pmlb/all_summary_stats.tsv"
dataset_df = pd.read_csv(cached_path.cached_path(path_to_data_summary), sep="\t")

classification_datasets = dataset_df[
    # (dataset_df["n_binary_features"] == dataset_df["n_features"])
    (dataset_df["task"] == "classification")
    & (dataset_df["n_classes"] == 2)
    & (dataset_df["n_features"] <= 100)
    & (dataset_df["n_instances"] <= 1000)
]["dataset"]

print(len(classification_datasets))

models = ['DT']


number_of_cv_folds = 5
random_state = 42

cv = StratifiedKFold(number_of_cv_folds, random_state=random_state, shuffle=True)

res = []
for dataset_index, classification_dataset in enumerate(classification_datasets[::-1][:]):
    
    print(f"{classification_dataset} ({dataset_index + 1}/{len(classification_datasets) + 1})")
    X, y = fetch_data(classification_dataset, return_X_y=True)
    if y.max() != 1 or y.min() != 0:
        for wanted, actual in enumerate(np.unique(y)):
            y[y==actual] = wanted
        
    imb_ratio = np.bincount(y).max() / np.bincount(y).min()
    print(f"{X.shape} with ratio : {imb_ratio:.4f}\n")
    
    # X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=random_state)
    
    for model_name in models:
        time_s = time.time()
        if model_name == 'DT':
            clf = DecisionTreeClassifier(random_state=random_state)
            # clf.fit(X_train, y_train)
            # y_probas = clf.predict_proba(X_test)
            # y_pred = np.argmax(y_probas, axis=1)
            y_pred = cross_val_predict(clf, X, y, cv=cv).astype(int)
        
        
        elif model_name == 'knn-dt':
            pass
            #y_pred = np.concatenate(y_pred)

        
        
        acc = accuracy_score(y, y_pred)
        (prec, rec, f1, sup) = precision_recall_fscore_support(
            y, y_pred, average="binary"
        )
            
        
        print(model_name)    
        print(classification_report(y, y_pred))
        time_end = time.time() - time_s
        res.append((classification_dataset, imb_ratio, model_name, time_end, acc, prec, rec, f1))
        
res = pd.DataFrame(res, columns=['dataset', 'dataset_class_imb', 'model', 'time', 'acc', 'pr', 'rec', 'f1'])
# res.sort_values('f1', ascending=False)

# Step 2: Sort each group by 'f1'
sorted_df = res.groupby('dataset').apply(lambda x: x.sort_values(by='f1', ascending=False)).reset_index(drop=True)

# Step 3: Assign ranks within each group
sorted_df['rank'] = sorted_df.groupby('dataset').cumcount() + 1

# Step 4: Calculate mean rank for each model across all datasets
mean_ranks = sorted_df.groupby('model')['rank'].mean().reset_index().sort_values(by='rank')

print(mean_ranks)
            
    #break

63
xd6 (1/64)
(973, 9) with ratio : 2.0217

DT
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       651
           1       0.99      1.00      1.00       322

    accuracy                           1.00       973
   macro avg       1.00      1.00      1.00       973
weighted avg       1.00      1.00      1.00       973

wdbc (2/64)
(569, 30) with ratio : 1.6840

DT
              precision    recall  f1-score   support

           0       0.92      0.93      0.93       357
           1       0.88      0.87      0.87       212

    accuracy                           0.91       569
   macro avg       0.90      0.90      0.90       569
weighted avg       0.91      0.91      0.91       569

vote (3/64)
(435, 16) with ratio : 1.5893

DT
              precision    recall  f1-score   support

           0       0.97      0.94      0.95       267
           1       0.91      0.95      0.93       168

    accuracy                           0.94

KeyboardInterrupt: 

In [40]:
import os
import random
import numpy as np
import torch

def set_all_seeds(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    # os.environ('PYTHONHASHSEED') = str(seed)
set_all_seeds(42)

In [199]:
from sklearn.metrics import f1_score
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import load_breast_cancer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import numpy as np

# Load and preprocess the dataset
# data = load_breast_cancer()
# X = data.data
# y = data.target

X, y = fetch_data("saheart", return_X_y=True)
if y.max() != 1 or y.min() != 0:
    for wanted, actual in enumerate(np.unique(y)):
        y[y==actual] = wanted


scaler = StandardScaler()
X = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

# Define the neural network model
class SimpleNN(nn.Module):
    def __init__(self, input_dim, embedding_dim):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(input_dim, 64)
        self.fc2 = nn.Linear(64, embedding_dim)
        self.fc3 = nn.Linear(embedding_dim, 1)
        self.relu = nn.ReLU()
    
    def forward(self, x):
        x = self.relu(self.fc1(x))
        embedding = self.relu(self.fc2(x))
        output = torch.sigmoid(self.fc3(embedding))
        return embedding, output

# # Function to compute contrastive loss using the entire dataset
# def contrastive_loss(embeddings, labels, margin=1.0, k=5):
#     batch_size = embeddings.size(0)
#     loss = 0.0
    
#     # Compute pairwise distances for all samples
#     distances = torch.cdist(embeddings, embeddings)  # Compute all pairwise distances
    
#     for i in range(batch_size):
#         anchor_label = labels[i]
#         anchor_distances = distances[i]

#         # Get indices of same-class and different-class samples
#         same_class_mask = (labels == anchor_label)
#         different_class_mask = (labels != anchor_label)

#         # Extract distances to same-class and different-class neighbors
#         same_class_distances = anchor_distances[same_class_mask]
#         different_class_distances = anchor_distances[different_class_mask]

#         # Sort distances and pick top-k
#         positive_top_k = torch.sort(same_class_distances)[0][:k]  # Closest same-class points
#         negative_top_k = torch.sort(different_class_distances)[0][:k]  # Closest different-class points

#         # Compute contrastive loss
#         positive_loss = torch.mean(positive_top_k)  # Minimize distances to same-class neighbors
#         negative_loss = torch.mean(torch.relu(margin - negative_top_k))  # Maximize distances to different-class neighbors
        
#         loss += positive_loss + negative_loss
    
#     return loss / batch_size


def contrastive_loss(embeddings, labels, margin=1, k=5):
    # Compute all pairwise distances
    distances = torch.cdist(embeddings, embeddings)  # (N, N) distance matrix

    # Create masks for same-class and different-class pairs
    labels_exp = labels.view(-1, 1)  # Reshape for broadcasting
    same_class_mask = labels_exp == labels_exp.T  # (N, N)
    different_class_mask = ~same_class_mask  # Inverse mask

    # Set diagonal to False (ignore self-pairs)
    same_class_mask.fill_diagonal_(False)
    different_class_mask.fill_diagonal_(False)

    # Extract distances for same-class and different-class
    same_class_distances = distances.clone()
    same_class_distances[~same_class_mask] = float('inf')  # Mask out non-same-class distances

    different_class_distances = distances.clone()
    different_class_distances[~different_class_mask] = float('inf')  # Mask out non-different-class distances

    # Get top-k closest distances for same-class and different-class
    positive_top_k, _ = torch.topk(same_class_distances, k, largest=False, dim=1)  # (N, k)
    negative_top_k, _ = torch.topk(different_class_distances, k, largest=False, dim=1)  # (N, k)

    # Compute loss
    positive_loss = positive_top_k.mean()
    negative_loss = torch.relu(margin - negative_top_k).mean() / margin
    total_loss = positive_loss + negative_loss#torch.relu((margin + negative_top_k) - positive_top_k.mean()).mean()/margin#
    return total_loss




# Initialize model, optimizer, and loss function
input_dim = X_train.shape[1]
embedding_dim = 64
model = SimpleNN(input_dim, embedding_dim)
optimizer = optim.Adam(model.parameters(), lr=0.0001)
criterion = nn.BCELoss()

# Training loop (without batching for now)
num_epochs = 10000
print_every = int(num_epochs/10)
best_score = -1
best_epoch = -1
patience_init = int(print_every/5)

weight_on_cl = 0.2
k = 3
margin = 1

for epoch in range(num_epochs):
    model.train()
    
    optimizer.zero_grad()
    
    embeddings, outputs = model(X_train_tensor)
    y_train_tensor_unsq = y_train_tensor.view(-1, 1)

    # Compute losses
    class_loss = criterion(outputs, y_train_tensor_unsq)
    contrast_loss = contrastive_loss(embeddings, y_train_tensor, margin=margin, k=k) # torch.Tensor([0])#
    loss = class_loss + weight_on_cl*contrast_loss

    loss.backward()
    optimizer.step()
    
    if (epoch + 1) % print_every == 0 and epoch + 1 >= print_every:
        print(f"Epoch {epoch+1}/{num_epochs}, Classification Loss: {class_loss.item():.4f}, Contrastive Loss: {contrast_loss.item():.4f}")
        # Evaluation
        model.eval()
        with torch.no_grad():
            _, test_outputs = model(X_test_tensor)
            test_predictions = (test_outputs.numpy() > 0.5).astype(int)
            # accuracy = np.mean(test_predictions == y_test.reshape(-1, 1))
            # print(f"Test Accuracy: {accuracy * 100:.2f}%")
            f1 = f1_score(y_test, test_predictions, average='macro')
            if f1 > best_score:
                best_score = f1
                best_epoch =  epoch + 1
                patience = patience_init
            else:
                patience -= 1
        if loss.item() <= 0.1 or patience == 0 :
            print(f"\nBreaking because patience = {patience} and Loss = {loss.item()}\n")
            break

print(f"Best Performance (on Test) -- F1: {f1:.4f} @ epoch {best_epoch}\n")
# Evaluation
model.eval()
with torch.no_grad():
    test_embeddings, test_outputs = model(X_test_tensor)
    test_predictions = (test_outputs.numpy() > 0.5).astype(int)
    # accuracy = np.mean(test_predictions == y_test.reshape(-1, 1))
    # print(f"Test Accuracy: {accuracy * 100:.2f}%")
    print(classification_report(y_test, test_predictions))
    

    train_embeddings, _ = model(X_train_tensor)
    dist = torch.cdist(test_embeddings, train_embeddings)
    knn_pred = (y_train[dist.argsort(axis=1)[:,:k].numpy()].mean(axis=1) >= 0.5).astype(int)
    # accuracy = np.mean(test_predictions == y_test.reshape(-1, 1))
    # print(f"Test Accuracy: {accuracy * 100:.2f}%")
    print(f'\nKNN Pred:\n')
    print(classification_report(y_test, knn_pred))


Epoch 1000/10000, Classification Loss: 0.4412, Contrastive Loss: 0.7366
Epoch 2000/10000, Classification Loss: 0.3134, Contrastive Loss: 0.6338
Epoch 3000/10000, Classification Loss: 0.1759, Contrastive Loss: 0.4489
Epoch 4000/10000, Classification Loss: 0.0852, Contrastive Loss: 0.2450
Epoch 5000/10000, Classification Loss: 0.0396, Contrastive Loss: 0.1304

Breaking because patience = 196 and Loss = 0.06566528975963593

Best Performance (on Test) -- F1: 0.6571 @ epoch 1000

              precision    recall  f1-score   support

           0       0.74      0.78      0.76        59
           1       0.58      0.53      0.55        34

    accuracy                           0.69        93
   macro avg       0.66      0.65      0.66        93
weighted avg       0.68      0.69      0.68        93


KNN Pred:

              precision    recall  f1-score   support

           0       0.76      0.80      0.78        59
           1       0.61      0.56      0.58        34

    accuracy     

In [18]:
Epoch 2750/5000, Classification Loss: 0.0973, Contrastive Loss: 2.4133

Breaking because patience = 49 and Loss = 0.09729014337062836

Best Performance (on Test) -- F1: 0.6613 @ epoch 200

              precision    recall  f1-score   support

           0       0.74      0.81      0.77        59
           1       0.61      0.50      0.55        34

    accuracy                           0.70        93
   macro avg       0.67      0.66      0.66        93
weighted avg       0.69      0.70      0.69        93


KNN Pred:

              precision    recall  f1-score   support

           0       0.72      0.78      0.75        59
           1       0.55      0.47      0.51        34

    accuracy                           0.67        93
   macro avg       0.64      0.63      0.63        93
weighted avg       0.66      0.67      0.66        93

False